Option 1: Collect the all projects with at least 5 releases using pandas DataFrames.

In [1]:
import json
import pandas as pd
import glob
import os
import numpy as np
from scipy.stats import gmean, kurtosis, skew
import datetime

languages = ["go", "java", "python", "ruby"]
project_details_folder = "project_details"
readmes_folder = "readmes"
releases_folder = "releases"

all_language_releases = []

for language in languages:

    total_projects = 0
    total_files = 0
    total_releases = 0
    total_readmes = 0
    more_than_5_releases = 0
    more_than_5_real_releases = 0

    # Grab all the files from within the folder, analyze the releases
    files = glob.glob(f"{releases_folder}/{language}/*_releases.jsonl")

    for file in files:

        total_files+=1
        owner, repo_name = file.replace("_releases.jsonl", "").split(f"{releases_folder}/{language}\\")[1].split("_", 1)

        # Dataframe of all the releases
        all_releases = pd.read_json(file, lines=True).sort_values(by="created_at")
        # all_releases.sort_values(by=['created_at'])

        # Find the index of all prereleases then drop them from the dataframe - giving us real releases
        real_releases = all_releases.copy(deep=True)
        indexPrereleases = real_releases[ (all_releases['prerelease'] == True)].index
        real_releases.drop(indexPrereleases, inplace=True)

        #display(real_releases)

        if len(all_releases.index) >= 5:
            more_than_5_releases += 1

        # If there are less than 5 releases, do not include this repository
        if len(real_releases.index) >= 5:
            more_than_5_real_releases+= 1
            
            real_release_date_diffs = real_releases["created_at"].diff().apply(lambda x: x/np.timedelta64(1, "D")).fillna(0).tolist()[1:]

            #df_all_releases = pd.DataFrame.from_records(all_releases)
            #df_real_releases = pd.DataFrame.from_records(real_releases)

            #release_dates_diffs = df_real_releases["created_at"].diff().apply(lambda x: x/np.timedelta64(1, "D")).fillna(0).tolist()[1:]
            release_dates_diffs_min = min(real_release_date_diffs) # TODO: Somehow there are negative date differences? Should check this.
            release_dates_diffs_max = max(real_release_date_diffs)
            release_dates_diffs_mean = np.mean(real_release_date_diffs)

            try:
                release_dates_diffs_gmean = gmean(real_release_date_diffs)
            except:
                release_dates_diffs_gmean = None
            try:
                release_dates_diffs_skewness = skew(real_release_date_diffs)
            except:
                release_dates_diffs_skewness = None
            try:
                release_dates_diffs_kurtosis = kurtosis(real_release_date_diffs)
            except:
                release_dates_diffs_kurtosis = None
            try:    
                release_dates_diffs_cv = np.std(real_release_date_diffs)/release_dates_diffs_mean
            except:
                release_dates_diffs_cv = None

            #release_dates = [date[:-6] for date in real_releases["created_at"].astype(str).tolist()] "release_dates": release_dates,
            real_release_summary = {"owner": owner, "repo": repo_name, "release_count": len(real_releases), "language": language, 
                             "release_dates_diffs": real_release_date_diffs, 
                            "release_dates_diffs_min": release_dates_diffs_min, "release_dates_diffs_max": release_dates_diffs_max, 
                            "release_dates_diffs_mean": release_dates_diffs_mean, "release_dates_diffs_gmean": release_dates_diffs_gmean, 
                            "release_dates_diffs_skewness": release_dates_diffs_skewness, 
                            "release_dates_diffs_kurtosis": release_dates_diffs_kurtosis, "release_dates_diffs_cv": release_dates_diffs_cv}
            all_language_releases.append(real_release_summary)

    print(total_files)
    print(f'{language} has {more_than_5_releases} projects, with more than 5 releases total.')
    print(f'{language} has {more_than_5_real_releases} projects, with more than 5 releases that are not prereleases.')

c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppD

593
go has 467 projects, with more than 5 releases total.
go has 447 projects, with more than 5 releases that are not prereleases.


c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppD

1317
java has 999 projects, with more than 5 releases total.
java has 950 projects, with more than 5 releases that are not prereleases.


c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppD

2330
python has 1621 projects, with more than 5 releases total.
python has 1544 projects, with more than 5 releases that are not prereleases.


c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\Users\Nathan\AppD

457
ruby has 291 projects, with more than 5 releases total.
ruby has 287 projects, with more than 5 releases that are not prereleases.


c:\Users\Nathan\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:279: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [2]:
releases = pd.DataFrame(all_language_releases, columns=all_language_releases[0].keys())

display(releases)

,owner,repo,release_count,language,release_dates_diffs,release_dates_diffs_min,release_dates_diffs_max,release_dates_diffs_mean,release_dates_diffs_gmean,release_dates_diffs_skewness,release_dates_diffs_kurtosis,release_dates_diffs_cv
0,8treenet,freedom,23,go,"[15.05900462962963, 8.676886574074073, 7.09667...",0.085775,278.917766,35.677011,13.702392,3.155394,9.974249,1.662474
1,a8m,documentdb,5,go,"[111.67515046296296, 315.0367939814815, 31.587...",31.587199,664.260185,280.639832,164.832100,0.620353,-1.155636,0.870864
2,abutaha,aws-es-proxy,7,go,"[92.51953703703704, 659.0286111111111, 498.567...",61.294468,659.028611,287.901985,202.390270,0.563902,-1.254768,0.773000
3,achannarasappa,ticker,47,go,"[3.146863425925926, 0.01, 0.5738773148148149, ...",0.002836,103.541759,12.533555,1.311554,2.333644,4.266249,2.075749
4,adjust,rmq,5,go,"[4.130902777777778, 2.0283680555555557, 161.80...",2.028368,161.802025,47.313001,13.034412,1.112311,-0.702475,1.405982
...,...,...,...,...,...,...,...,...,...,...,...,...
3223,xotahal,fastlane-plugin-semantic_release,23,ruby,"[0.05168981481481481, 67.12336805555556, 0.087...",0.001377,337.645220,57.694966,2.960668,1.821206,2.295284,1.584347
3224,yoshoku,rumale,10,ruby,"[50.351226851851855, 12.845046296296296, 36.02...",12.845046,324.441875,65.737301,36.030668,2.249610,3.440039,1.428417
3225,yujinakayama,guard-rubocop,12,ruby,"[3.795150462962963, 6.819120370370371, 44.1720...",3.795150,2754.523414,277.390703,30.969192,2.842263,6.087186,2.824980
3226,yuki24,rambulance,17,ruby,"[2.067662037037037, 75.40505787037037, 12.6325...",1.480775,513.022454,174.705454,54.857229,0.827077,-0.919225,1.050759


In [1]:
from plotly.express import box

fig = box(releases, x="language", y="release_dates_diffs_gmean", color="language", 
          labels={"language": "Language", "release_dates_diffs_gmean": "Geometric release interval mean"}, height=500, log_y=True)

fig.update_xaxes(tickangle=30).update_traces(boxpoints=False) 

fig.show()

NameError: name 'releases' is not defined

In [4]:
from plotly.express import violin

fig = violin(releases, x="language", y="release_dates_diffs_gmean", color="language", 
             labels={"language": "Language", "release_dates_diffs_gmean": "Geometric release interval mean"}, height=500, log_y=False)

fig.update_xaxes(tickangle=30)

fig.show()

In [5]:
from plotly.express import box

fig = box(releases, x="language", y="release_dates_diffs_cv", color="language", 
          labels={"language": "Language", "release_dates_diffs_cv": "Release interval coefficient of variation"}, height=500, log_y=True)

fig.update_xaxes(tickangle=30)

fig.show()